# IMPORT LIBARY


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk

# LOAD DATASETS


In [ ]:
data_path = "datasets/supermarket_sales.csv"

df = pd.read_csv(data_path)


# melihat 5 data teratas
df.head(5)

In [ ]:
df.shape

In [ ]:
# melihat tipe data pada tiap kolom header
df.info()

In [ ]:
# mendeskirpsikan isi dari datasets
df.describe()

# DATA PREPROCESSING


In [ ]:
df.isna().sum()
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df = df.drop('gross margin percentage', axis=1)


In [ ]:
# mengecek struktur table setelah kolom gross margin precentage di drop
df.head(10)

In [ ]:
label, feature = df.shape
# melihat jumlah feature
print(feature)

# melihat jumlah label
print(label)

In [ ]:
# meliaht jumlah data unique
df.nunique()

In [ ]:
unique_counts = df.nunique()
unique_df = unique_counts.reset_index()
unique_df.columns = ["feature", "jumlah_unique"]

plt.figure(figsize=(10, 6))

sns.barplot(
    data=unique_df,
    x="feature",
    y="jumlah_unique"
)

plt.title("Jumlah Data Unik Pada Setiap Feature")
plt.xlabel("Feature")
plt.ylabel("Jumlah Unique Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# EXPLORATORY DATA ANALYS


In [ ]:
num_cols = ['Unit price', 'Quantity', 'Total', 'Rating']

plt.figure(figsize=(12, 8))
for i, col in enumerate(num_cols, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribusi {col}')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(
    x='City',
    y='Total',
    data=df,
    estimator=sum
)

plt.title("Total Penjualan per Kota")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(
    y='Product line',
    x='Total',
    data=df,
    estimator=sum
)

plt.title("Total Penjualan per Product Line")
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(
    x='Payment',
    data=df
)

plt.title("Jumlah Transaksi per Metode Pembayaran")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.scatterplot(
    x='Quantity',
    y='Total',
    data=df
)

plt.title("Hubungan Quantity terhadap Total")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(
    x='Gender',
    y='Rating',
    data=df
)

plt.title("Distribusi Rating Berdasarkan Gender")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(
    x='Customer type',
    y='Total',
    data=df,
    estimator=sum
)

plt.title("Total Penjualan Berdasarkan Tipe Customer")
plt.show()


In [ ]:
# membuat diagram korelasi antar data dalamx dengan data dalam y menggunakan heatmap
x_vars = ['Unit price', 'Quantity']
y_vars = ['Total']

sns.pairplot(
    df,
    x_vars=x_vars,
    y_vars=y_vars,
    height=4,
    aspect=1,
    kind='scatter'
)

plt.suptitle("Korelasi data X terhadap data Y", y=1.02)
plt.show()

In [ ]:
df_numeric = df.select_dtypes(include=['int64', 'float64'])

plt.figure(figsize=(10, 8))
sns.heatmap(
    df_numeric.corr(),
    annot=True,
    cmap='coolwarm',
    fmt='.2f'
)

plt.title("Heatmap Korelasi Data Numerik")
plt.show()

In [ ]:
# normalisasi nama kolom agar mudah di gunakan
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.columns

In [ ]:
# arr_to_encode = ['branch', 'city', 'customer_type', 'gender', 'product_line', 'payment'] 
# for col in arr_to_encode:
#     print(f"Nilai unik di {col}: {df[col].unique()}")

# # Contoh Encode Manual Sederhana
# mapping_branch = {'A': 0, 'B':1, 'C':2}
# df['branch'] = [mapping_branch[x] for x in df['branch']]

# mapping_city = {'Yangon': 0, 'Naypyitaw': 1, 'Mandalay': 2}
# df['city'] = [mapping_city[x] for x in df['city']]

# mapping_customer_type = {'Member': 0, 'Normal': 1}
# df['customer_type'] = [mapping_customer_type[x] for x in df['customer_type']]

# mapping_gender = {'Female': 0, 'Male': 1}
# df['gender'] = 

# # Lakukan hal yang sama untuk Branch dan Customer type

# LINEAR REGRETION ALGORYTHM


In [ ]:
# split data untuk train dan data untuk test
def train_test_split(data, test_size = 0.2, random_state = 42, target_columns = 'total'): # random state untuk menghitung jumlah data yang akan di test, random_state untuk merekam data yang sudah kita acak
    # spliting data test dan data train yang akan di gunakan
    test_data = data.sample(frac=test_size, random_state=random_state)
    train_data = data.drop(test_data.index)

    # mengambil data test
    X_test = test_data.drop(columns=[target_columns])
    y_test = test_data[target_columns]

    # mengambil data train
    X_train = train_data.drop(columns=[target_columns])
    y_train = train_data[target_columns]

    # return value data
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(df)

In [ ]:
class LinearRegresion:
    def __init__(self, learn_r = 0.001, iters = 300):
        self.learn_r = learn_r
        self.iters = iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X = np.array(X, dtype='float')
        y = np.array(y, dtype='float')

        feats, label = X.shape

        self.weights = np.zeros(label) # init weights dari titik 0 menggunakan np.zeros
        self.bias = 0 # init bias titik netral dari garis yang akan di tarik

        for x in range(self.iters):
            y_prediction = np.dot(X, self.weights) + self.bias

            dw = (1 / feats) * np.dot(X.T, (y_prediction - y))
            db = (1 / feats) * np.sum(y_prediction - y)

            self.weights -= self.learn_r * dw
            self.bias -= self.learn_r * db
            
            if i % 100 == 0:
                mse = np.mean((y_prediction - y)**2)
                print(f"Iterasi {i}: Error (MSE) = {mse}")

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [ ]:
my_features = ['unit_price', 'quantity', 'rating'] # menyesuaikan dengan inputan data dari gui

X_train_final = X_train[my_features].values
y_train_final = y_train.values

X_test_final = X_test[my_features].values

x_min = X_train_final.min(axis=0)
x_max = X_train_final.max(axis=0)
X_train_scaled = (X_train_final - x_min) / (x_max - x_min)
X_test_scaled = (X_test_final - x_min) / (x_max - x_min)

ln = LinearRegresion(learn_r=0.01, iters=2000)
ln.fit(X_train_scaled, y_train_final)

raw_predictions = ln.predict(X_test_scaled)

data_total = y_train.values 

batas_rendah = np.percentile(data_total, 33)
batas_sedang = np.percentile(data_total, 66)

def classify_auto(value, b_rendah, b_tinggi):
    if value < b_rendah:
        return "Rendah"
    elif value <= b_tinggi:
        return "Sedang"
    else:
        return "Tinggi"

y_pred_labels = [classify_auto(p, batas_rendah, batas_sedang) for p in raw_predictions]

y_test_labels = [classify_auto(y, batas_rendah, batas_sedang) for y in y_test]

y_test_labels[:5]

In [ ]:
y_test_array = np.array(y_test, dtype="float")
y_pred_array = np.array(raw_predictions, dtype="float")

plt.figure(figsize=(6, 6))
plt.scatter(y_test_array, y_pred_array, alpha=0.6)

min_val = min(y_test_array.min(), y_pred_array.min())
max_val = max(y_test_array.max(), y_pred_array.max())
plt.plot([min_val, max_val], [min_val, max_val], "r--", label="y = x")

plt.xlabel("Total Aktual")
plt.ylabel("Total Prediksi")
plt.title("Perbandingan Total Aktual vs Prediksi (Linear Regresi)")
plt.legend()
plt.tight_layout()
plt.show()

residuals = y_test_array - y_pred_array

plt.figure(figsize=(6, 4))
plt.scatter(y_test_array, residuals, alpha=0.6)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Total Aktual")
plt.ylabel("Residual (Aktual - Prediksi)")
plt.title("Plot Residual Regresi Linear")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.histplot(residuals, kde=True)
plt.title("Distribusi Residual Regresi Linear")
plt.xlabel("Residual")
plt.tight_layout()
plt.show()

# GUI 

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox


class GUI:
    def __init__(self, root, tabCtrl):
        self.root = root
        self.root.geometry("750x450")
        self.tabCtrl = tabCtrl
        self.root.title("GUI LINEAR REGRESI")
        self.tabCtrl.pack(expand=True, fill="both")
        self.bg = "#e7eaeb"

        self.layouts()

    def validate_number(self, value):
        if value == "":
            return True
        try:
            float(value)
            return True
        except ValueError:
            return False

    def layouts(self):
        tab1 = tk.Frame(self.tabCtrl, bg=self.bg)
        self.tabCtrl.add(tab1, text="Pengujian Algoritma")

        header_frame = tk.Frame(tab1, bg="#f5f6f7", bd=2, relief="groove")
        header_frame.pack(pady=15, padx=20, fill="x")

        tk.Label(
            header_frame,
            text="GUI PENGUJI ALGORITMA REGRESI LINEAR",
            bg="#f5f6f7",
            font=("Segoe UI", 18, "italic"),
            fg="#333",
        ).pack(padx=15, pady=10)

        # ===== FORM FRAME (STRUKTUR ASLI) =====
        form_frame = tk.Frame(tab1, bg=self.bg)
        form_frame.pack(pady=15, padx=30, fill="x", side="left")

        vcmd = (self.root.register(self.validate_number), "%P")

        # ===== UNIT PRICE =====
        unit_frame = tk.Frame(form_frame, bg=self.bg)
        unit_frame.pack(fill="x", pady=8)

        tk.Label(unit_frame, text="Unit Price", bg=self.bg, font=("Segoe UI", 10)).pack(
            anchor="w"
        )

        self.unit_entry = tk.Entry(
            unit_frame, font=("Segoe UI", 10), validate="key", validatecommand=vcmd
        )
        self.unit_entry.pack(fill="x", pady=(4, 0))

        # ===== QUANTITY =====
        qty_frame = tk.Frame(form_frame, bg=self.bg)
        qty_frame.pack(fill="x", pady=8)

        tk.Label(qty_frame, text="Quantity", bg=self.bg, font=("Segoe UI", 10)).pack(
            anchor="w"
        )

        self.qty_entry = tk.Entry(
            qty_frame, font=("Segoe UI", 10), validate="key", validatecommand=vcmd
        )
        self.qty_entry.pack(fill="x", pady=(4, 0))

        # ===== RATING =====
        rating_frame = tk.Frame(form_frame, bg=self.bg)
        rating_frame.pack(fill="x", pady=8)

        tk.Label(rating_frame, text="Rating", bg=self.bg, font=("Segoe UI", 10)).pack(
            anchor="w"
        )

        self.rating_entry = tk.Entry(
            rating_frame, font=("Segoe UI", 10), validate="key", validatecommand=vcmd
        )
        self.rating_entry.pack(fill="x", pady=(4, 0))

        # ===== BUTTON =====
        btn_frame = tk.Frame(form_frame, bg=self.bg)
        btn_frame.pack(fill="x", pady=(20, 0))

        self.predict_btn = tk.Button(
            btn_frame,
            text="Prediksi",
            font=("Segoe UI", 10, "bold"),
            bg="#4a90e2",
            fg="white",
            relief="flat",
            padx=60,
            pady=6,
        )
        self.predict_btn.pack(anchor="w", side="left")

        self.restart_btn = tk.Button(
            btn_frame,
            text="Restart",
            font=("Segoe UI", 10, "bold"),
            bg="#4ae293",
            fg="white",
            relief="flat",
            padx=60,
            pady=6,
        )
        self.restart_btn.pack(anchor="w", side="right")

        output_frame = tk.Frame(tab1, bg=self.bg)
        output_frame.pack(pady=10, padx=40, side="left")

        tk.Label(
            output_frame,
            text="Nilai Total Prediksi : ",
            font=("Segoe UI", 15, "bold"),
            bg=self.bg,
        ).pack(anchor="w", pady=5)

        self.result_nilai = tk.Label(
            output_frame,
            text="-",
            font=("Segoe UI", 10, "bold"),
            bg="#e7eaeb",
            fg="#333",
        )
        self.result_nilai.pack(anchor="w", pady=5)

        tk.Label(
            output_frame,
            text="Kategori Penjualan : ",
            font=("Segoe UI", 15, "bold"),
            bg=self.bg,
        ).pack(anchor="w", pady=5)

        self.result_kategori = tk.Label(
            output_frame,
            text="-",
            font=("Segoe UI", 10, "bold"),
            bg="#e7eaeb",
            fg="#333",
        )
        self.result_kategori.pack(anchor="w", pady=5)

        # tab visualisasi data
        tab2 = tk.Frame(self.tabCtrl, bg=self.bg)
        self.tabCtrl.add(tab2, text="Visualisasi Data")
        
    def predict(self):
        try : 
            unit = float(self.unit_entry.get())
            quantity = float(self.qty_entry.get())
            rating = float(self.rating_entry.get())
            
            # self.linear = #fungsi dummy
            
            
        except ValueError:
            messagebox.showerror("error", "Input harus berupa angka!")

if __name__ == "__main__":
    root = tk.Tk()
    tabCtrl = ttk.Notebook(root)
    app = GUI(root, tabCtrl)
    root.mainloop()